## Step 1.
Create a .env file with the following:
~~~bash
export STRAVA_CLIENT_ID="xxxx"
export STRAVA_CLIENT_SECRET="xxxx"
~~~

In [18]:
%load_ext dotenv
%dotenv

import pandas as pd
import plotly.express as px
import os, requests
import time
import pickle

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Step 2.
Run the code and Click on the below link, you will get redirected. Copy and paste the code from the redirect link and fill in 'code = '''

the redirect url will look like: http://127.0.0.1:5000/authorization?state=&code=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&scope=read,activity:read_all,profile:read_all,read_all

you may need to click on the link bar to see the extended url

In [13]:
from stravalib.client import Client

client = Client()
authorize_url = client.authorization_url(
    client_id=os.environ['STRAVA_CLIENT_ID'], redirect_uri="http://localhost:8282/authorized"
)

url = client.authorization_url(
    client_id=os.environ['STRAVA_CLIENT_ID'], 
    redirect_uri='http://127.0.0.1:5000/authorization', 
    scope=['read_all','profile:read_all','activity:read_all'],
)

url

'https://www.strava.com/oauth/authorize?client_id=137744&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fauthorization&approval_prompt=auto&scope=read_all%2Cprofile%3Aread_all%2Cactivity%3Aread_all&response_type=code'

In [14]:
CODE = '3a056f1bf3dced2c9b5eb2ffb82fbdabd92c56d7'
access_token = client.exchange_code_for_token(
    client_id=os.environ['STRAVA_CLIENT_ID'],
    client_secret=os.environ['STRAVA_CLIENT_SECRET'], 
    code=CODE,
)

with open('./access_token.pickle', 'wb') as f:
    pickle.dump(access_token, f)

No rates present in response headers


## Step 3.
Load in access token

In [22]:
with open('./access_token.pickle', 'rb') as f:
    access_token = pickle.load(f)
    
print('Latest access token read from file:')
access_token

Latest access token read from file:


{'access_token': 'aa718201eb30b4f89190b16d0e287d7e5c89121c',
 'refresh_token': 'f8f5b2b01e5b8e65ef7534bb790cd1afba070563',
 'expires_at': 1729110698}

In [23]:
if time.time() > access_token['expires_at']:
    print('Token has expired, will refresh')
    refresh_response = client.refresh_access_token(client_id=MY_STRAVA_CLIENT_ID, client_secret=MY_STRAVA_CLIENT_SECRET, refresh_token=access_token['refresh_token'])
    access_token = refresh_response
    with open('./access_token.pickle', 'wb') as f:
        pickle.dump(refresh_response, f)
    print('Refreshed token saved to file')
    client.access_token = refresh_response['access_token']
    client.refresh_token = refresh_response['refresh_token']
    client.token_expires_at = refresh_response['expires_at']
        
else:
    print('Token still valid, expires at {}'
          .format(time.strftime("%a, %d %b %Y %H:%M:%S %Z", time.localtime(access_token['expires_at']))))
    client.access_token = access_token['access_token']
    client.refresh_token = access_token['refresh_token']
    client.token_expires_at = access_token['expires_at']

Token still valid, expires at Wed, 16 Oct 2024 16:31:38 EDT


## Step 4. Get Data

In [24]:
athlete = client.get_athlete()
print("Athlete's name is {} {}, based in {}, {}"
      .format(athlete.firstname, athlete.lastname, athlete.city, athlete.country))

Athlete's name is Bradford Gill, based in San Francisco, United States


In [29]:
activities = list(client.get_activities(limit=100))
activities[0:10]

[SummaryActivity(bound_client=<stravalib.client.Client object at 0x1260e86b0>, id=12665226865, achievement_count=0, athlete=MetaAthlete(bound_client=None, id=2033770, resource_state=1), athlete_count=1, average_speed=3.073, average_watts=366.3, comment_count=0, commute=False, device_watts=True, distance=16124.5, elapsed_time=6000, elev_high=52.6, elev_low=4.0, end_latlng=LatLon(root=[42.04542095772922, -72.59876718744636]), external_id='garmin_ping_378057902474', flagged=False, gear_id='g19532103', has_kudoed=False, hide_from_home=None, kilojoules=1922.0, kudos_count=16, manual=False, map=Map(id='a12665226865', polyline=None, summary_polyline='yzt_Gz{~yLQdDBJt@P^Nv@LdCr@Xh@TZDTAx@MpAEbAU~CSbBWrAc@zA}@xBShAGxA\\jCCZOVo@RoC\\sF^_ANgCDGHCTKjDMfAErAUbBGXEJo@AeCSsDg@sDYs@O}@Cs@XsA_@qC]aBKcDa@qDWSIq@i@eCs@wAM}A?qAYkAAi@EEC|B@~AZfA?f@FjDr@XLv@j@p@H|DZ~Dj@dCR|A\\pCGr@LrLjAfATHJC^s@nCa@xB[lAGt@BZFVVZr@b@f@f@f@^`B`BhGhFvBtBr@b@v@ZdAHjF`AfKzAfEv@jGv@FPEr@@t@QbAE`BK|Ae@zB@l@Rd@`Aj@|D|@vBZPJDN?fAFx

In [26]:
my_cols =['name',
          'start_date_local',
          'type',
          'distance',
          'moving_time',
          'elapsed_time',
          'total_elevation_gain',
          'elev_high',
          'elev_low',
          'average_speed',
          'max_speed',
          'average_heartrate',
          'max_heartrate',
          'start_latlng',]

In [33]:
data = []
for activity in activities:
    my_dict = dict(activity)
    data.append([activity.id]+[my_dict.get(x) for x in my_cols])
    
# Add id to the beginning of the columns, used when selecting a specific activity
my_cols.insert(0,'id')

In [34]:
df = pd.DataFrame(data, columns=my_cols)
df

,id,name,start_date_local,type,distance,moving_time,elapsed_time,total_elevation_gain,elev_high,elev_low,average_speed,max_speed,average_heartrate,max_heartrate,start_latlng
0,12665226865,Evening trails,2024-10-15 17:12:16+00:00,root='Run',16124.5,5247,6000,30.0,52.6,4.0,3.073,4.400,None,None,"root=[42.054893262684345, -72.5780063867569]"
1,12641047845,The whites are no joke…,2024-10-12 06:37:20+00:00,root='Run',51534.1,38358,46976,3733.0,1555.0,352.2,1.344,4.838,None,None,"root=[44.1816118452698, -71.39875140041113]"
2,12625073046,1000 miles on the year 🎉🥳🎉,2024-10-10 17:31:08+00:00,root='Run',9373.8,3062,3546,65.0,59.8,13.0,3.061,4.808,None,None,"root=[42.05529768951237, -72.57778208702803]"
3,12617170518,Afternoon Run,2024-10-09 16:05:50+00:00,root='Run',9539.5,2783,3807,15.0,42.0,31.4,3.428,6.054,None,None,"root=[42.33548945747316, -72.62153243646026]"
4,12605913498,Lithia springs foliage 🍁 🍂,2024-10-08 08:37:06+00:00,root='Run',10019.6,6023,6179,519.0,323.8,58.8,1.664,4.026,None,None,"root=[42.30529775843024, -72.52873160876334]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11842205465,Afternoon Mountain Bike Ride,2024-07-08 17:59:32+00:00,root='Ride',4370.2,1914,2504,158.0,1422.4,1382.2,2.283,7.282,None,None,"root=[38.58189267106354, -109.51928375288844]"
96,11841098247,It’s hot in Moab,2024-07-08 06:23:55+00:00,root='Ride',81540.6,25066,32097,1493.0,1787.0,1351.6,3.253,12.150,None,None,"root=[38.82998635992408, -109.76309403777122]"
97,11833516073,Afternoon Mountain Bike Ride,2024-07-07 13:16:54+00:00,root='Ride',23754.4,7462,8373,613.0,2870.8,2115.0,3.183,9.654,None,None,"root=[40.62071057036519, -111.49269699119031]"
98,11824067878,Morning Mountain Bike Ride,2024-07-06 09:09:55+00:00,root='Ride',36879.9,7147,10108,317.0,3015.8,1510.4,5.160,17.814,None,None,"root=[40.60672645457089, -111.55493513680995]"
